In [7]:
import pandas as pd
import os
import time
from time import mktime
from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
style.use('dark_background')

import re

path = os.getcwd()

In [15]:
statsPath = path+'/intraQuarter/_KeyStats'
stockList = [x[0].split('/')[-1] for x in os.walk(statsPath)]

with open('stocklist.txt', 'w') as f:
    for item in (sorted(stockList[1:])):
        f.write("%s\n" % item)

In [2]:
def Key_Stats(gather=["Total Debt/Equity",
                      'Trailing P/E',
                      'Price/Sales',
                      'Price/Book',
                      'Profit Margin',
                      'Operating Margin',
                      'Return on Assets',
                      'Return on Equity',
                      'Revenue Per Share',
                      'Market Cap',
                        'Enterprise Value',
                        'Forward P/E',
                        'PEG Ratio',
                        'Enterprise Value/Revenue',
                        'Enterprise Value/EBITDA',
                        'Revenue',
                        'Gross Profit',
                        'EBITDA',
                        'Net Income Avl to Common ',
                        'Diluted EPS',
                        'Earnings Growth',
                        'Revenue Growth',
                        'Total Cash',
                        'Total Cash Per Share',
                        'Total Debt',
                        'Current Ratio',
                        'Book Value Per Share',
                        'Cash Flow',
                        'Beta',
                        'Held by Insiders',
                        'Held by Institutions',
                        'Shares Short (as of',
                        'Short Ratio',
                        'Short % of Float',
                        'Shares Short (prior ']):
    
    statsPath = path+'/intraQuarter/_KeyStats'
    stockList = [x[0] for x in os.walk(statsPath)]
    
    df = pd.DataFrame(columns = ['Date',
                                 'Unix',
                                 'Ticker',
                                 'Price',
                                 'stock_p_change',
                                 'SP500',
                                 'sp500_p_change',
                                 'Difference',
                                 'DE Ratio',
                                 'Trailing P/E',
                                 'Price/Sales',
                                 'Price/Book',
                                 'Profit Margin',
                                 'Operating Margin',
                                 'Return on Assets',
                                 'Return on Equity',
                                 'Revenue Per Share',
                                 'Market Cap',
                                 'Enterprise Value',
                                 'Forward P/E',
                                 'PEG Ratio',
                                 'Enterprise Value/Revenue',
                                 'Enterprise Value/EBITDA',
                                 'Revenue',
                                 'Gross Profit',
                                 'EBITDA',
                                 'Net Income Avl to Common ',
                                 'Diluted EPS',
                                 'Earnings Growth',
                                 'Revenue Growth',
                                 'Total Cash',
                                 'Total Cash Per Share',
                                 'Total Debt',
                                 'Current Ratio',
                                 'Book Value Per Share',
                                 'Cash Flow',
                                 'Beta',
                                 'Held by Insiders',
                                 'Held by Institutions',
                                 'Shares Short (as of',
                                 'Short Ratio',
                                 'Short % of Float',
                                 'Shares Short (prior)',
                                 'Status'])

    
    sp500 = pd.read_csv('s&p500.csv')
    
    tickerList = []
    for dir in (stockList)[1:]:
        files = os.listdir(dir)
        ticker = dir.split('/')[-1]
        tickerList.append(ticker)
        
        stock_Start = False
        sp500_Start = False
        
        if len(files) > 0:
            for file in files:
                dateStamp = datetime.strptime(file, '%Y%m%d%H%M%S.html')
                unixTime = time.mktime(dateStamp.timetuple())
                
                filePath = dir+'/'+file
                # HTML source variable
                source = open(filePath,'r').read()
                # If Data exists in this format
                try:
                    value_list = []
                    # Foreach in gather list
                    for data in gather:
                        try:
                            regex = re.escape(data) + r'.*?(\d{1,8}\.\d{1,8}M?B?|N/A)%?</td>'
                            value = re.search(regex, source)
                            value = value.group(1)
                            
                            if 'B' in value:
                                value = float(value.replace('B',''))*1000000000
                            elif 'M' in value:
                                value = float(value.replace('M',''))*1000000
                            value_list.append(value)
                        except:
                            value = 'N/A'
                            value_list.append(value)

                    try:
                        stockPrice = float(source.split('</small><big><b>')[1].split('</b></big>')[0])
                    except:
                        try:
                            stockPrice = source.split('</small><big><b>')[1].split('</b></big>')[0]
                            stockPrice = re.search(r'\d{1,8}\.\d{1,8}', stockPrice)
                            stockPrice = float(stockPrice.group(0))
                            
                        except Exception as e:
                            stockPrice = source.split('<span class="time_rtq_ticker">')[1].split('</span>')[0]
                            stockPrice = re.search(r'\d{1,8}\.\d{1,8}', stockPrice)
                            stockPrice = float(stockPrice.group(0))        
                    
                    try:
                        # Not a weekend
                        sp500_date = datetime.fromtimestamp(unixTime).strftime('%Y-%m-%d')
                        row = sp500[sp500["Date"] == sp500_date]
                        sp500Value = float(row['Adj Close'])

                    except:
                        # If weekend, subtract 3 days from unixTime
                        sp500_date = datetime.fromtimestamp(unixTime-259200).strftime('%Y-%m-%d')
                        row = sp500[sp500["Date"] == sp500_date]
                        sp500Value = float(row['Adj Close'])
                    
                    
                    if stock_Start is False:
                        stock_Start = stockPrice
                    
                    if sp500_Start is False:
                        sp500_Start = sp500Value
                    
                    
                    stock_p_change = ((stockPrice - stock_Start)/stock_Start)*100
                    sp500_p_change = ((sp500Value - sp500_Start)/sp500_Start)*100     
                    
                    diff = stock_p_change - sp500_p_change
                    
                    if diff > 0:
                        status = 'Up'
                    else:
                        status = 'Down'
                    
                    if value_list.count('N/A') > 0:
                        pass
                    
                    else:
                        df = df.append({'Date':dateStamp,
                                        'Unix':unixTime,
                                        'Ticker':ticker,
                                        'Price':stockPrice,
                                        'stock_p_change':stock_p_change,
                                        'SP500':sp500Value,
                                        'sp500_p_change':sp500_p_change,
                                        'Difference':diff,
                                        'DE Ratio':value_list[0],
                                        'Trailing P/E':value_list[1],
                                        'Price/Sales':value_list[2],
                                        'Price/Book':value_list[3],
                                        'Profit Margin':value_list[4],
                                        'Operating Margin':value_list[5],
                                        'Return on Assets':value_list[6],
                                        'Return on Equity':value_list[7],
                                        'Revenue Per Share':value_list[8],
                                        'Market Cap':value_list[9],
                                        'Enterprise Value':value_list[10],
                                        'Forward P/E':value_list[11],
                                        'PEG Ratio':value_list[12],
                                        'Enterprise Value/Revenue':value_list[13],
                                        'Enterprise Value/EBITDA':value_list[14],
                                        'Revenue':value_list[15],
                                        'Gross Profit':value_list[16],
                                        'EBITDA':value_list[17],
                                        'Net Income Avl to Common ':value_list[18],
                                        'Diluted EPS':value_list[19],
                                        'Earnings Growth':value_list[20],
                                        'Revenue Growth':value_list[21],
                                        'Total Cash':value_list[22],
                                        'Total Cash Per Share':value_list[23],
                                        'Total Debt':value_list[24],
                                        'Current Ratio':value_list[25],
                                        'Book Value Per Share':value_list[26],
                                        'Cash Flow':value_list[27],
                                        'Beta':value_list[28],
                                        'Held by Insiders':value_list[29],
                                        'Held by Institutions':value_list[30],
                                        'Shares Short (as of':value_list[31],
                                        'Short Ratio':value_list[32],
                                        'Short % of Float':value_list[33],
                                        'Shares Short (prior)':value_list[34],
                                        'Status':status},
                                       ignore_index=True)
                except:
                    pass
    
#     for ticker in tickerList:
#         try:
#             plot = df[df['Ticker'] == ticker]
#             plot = plot.set_index(['Date'])
#             print(plot)
            
#             if plot['Status'][-1] == 'Down':
#                 color = 'r'
#             else:
#                 color = 'green'
            
#             plot['Difference'].plot(label=ticker, color = color)
#             plt.legend()
#         except:
#             pass
        
#     plt.show()
    
    save = 'Processed_Data.csv'
    df.to_csv(save)               

In [3]:
Key_Stats()

In [6]:
df = pd.read_csv('Processed_Data.csv')
df


,Unnamed: 0,Date,Unix,Ticker,Price,stock_p_change,SP500,sp500_p_change,Difference,DE Ratio,...,Book Value Per Share,Cash Flow,Beta,Held by Insiders,Held by Institutions,Shares Short (as of,Short Ratio,Short % of Float,Shares Short (prior),Status
0,0,2011-08-16 01:34:54,1.313439e+09,unp,93.85,-9.060078,1192.760010,-17.040974,7.980896,48.830,...,37.520000,5.314000e+10,1.140,0.24,85.10,4580000.0,1.600,0.90,4400000.0,Up
1,1,2013-04-23 03:57:01,1.366670e+09,unp,147.44,42.868217,1578.780029,9.807550,33.060667,48.950,...,43.060000,6.280000e+09,1.040,0.29,81.50,3650000.0,2.000,0.80,3880000.0,Up
2,2,2004-09-04 18:53:12,1.094304e+09,unp,57.24,-44.534884,1105.910034,-23.081576,-21.453307,0.652,...,48.473999,2.290000e+09,0.360,5.52,79.70,1850000.0,1.936,0.76,1430000.0,Down
3,3,2013-08-19 06:03:49,1.376872e+09,unp,157.93,53.032946,1646.060059,14.487021,38.545925,47.790,...,44.200000,6.600000e+09,1.080,0.30,81.20,3600000.0,2.400,0.80,4210000.0,Up
4,4,2004-06-21 18:44:53,1.087824e+09,unp,57.76,-44.031008,1130.300049,-21.385198,-22.645809,0.636,...,48.160999,2.420000e+09,0.361,5.53,78.85,2520000.0,1.542,1.03,1480000.0,Down
5,5,2012-04-24 04:51:29,1.335223e+09,unp,109.00,5.620155,1371.969971,-4.576535,10.196690,47.110,...,39.330000,5.980000e+09,1.270,0.30,83.60,6120000.0,2.000,1.30,6550000.0,Up
6,6,2012-03-19 15:36:23,1.332152e+09,unp,113.10,9.593023,1409.750000,-1.948853,11.541876,47.940,...,38.710000,5.870000e+09,1.270,0.30,83.60,5140000.0,1.700,1.00,5130000.0,Up
7,7,2012-01-02 08:04:30,1.325472e+09,unp,105.94,2.655039,1257.599976,-12.531214,15.186253,50.670,...,38.410000,5.720000e+09,1.290,0.24,85.10,4770000.0,1.800,1.00,4770000.0,Up
8,8,2004-12-14 06:56:14,1.102988e+09,unp,62.10,-39.825581,1203.380005,-16.302330,-23.523251,0.643,...,48.951000,2.370000e+09,0.326,0.84,79.90,2710000.0,2.348,1.05,2870000.0,Down
9,9,2013-04-25 00:40:45,1.366831e+09,unp,147.30,42.732558,1585.160034,10.251293,32.481265,48.950,...,43.060000,6.280000e+09,1.040,0.29,81.50,3650000.0,2.000,0.80,3880000.0,Up


In [5]:
sp500 = pd.read_csv('s&p500.csv')
print(sp500['Date'])

0       2000-09-19
1       2000-09-20
2       2000-09-21
3       2000-09-22
4       2000-09-25
5       2000-09-26
6       2000-09-27
7       2000-09-28
8       2000-09-29
9       2000-10-02
10      2000-10-03
11      2000-10-04
12      2000-10-05
13      2000-10-06
14      2000-10-09
15      2000-10-10
16      2000-10-11
17      2000-10-12
18      2000-10-13
19      2000-10-16
20      2000-10-17
21      2000-10-18
22      2000-10-19
23      2000-10-20
24      2000-10-23
25      2000-10-24
26      2000-10-25
27      2000-10-26
28      2000-10-27
29      2000-10-30
           ...    
4750    2019-08-08
4751    2019-08-09
4752    2019-08-12
4753    2019-08-13
4754    2019-08-14
4755    2019-08-15
4756    2019-08-16
4757    2019-08-19
4758    2019-08-20
4759    2019-08-21
4760    2019-08-22
4761    2019-08-23
4762    2019-08-26
4763    2019-08-27
4764    2019-08-28
4765    2019-08-29
4766    2019-08-30
4767    2019-09-03
4768    2019-09-04
4769    2019-09-05
4770    2019-09-06
4771    2019